In [1]:
import pandas as pd
treino = pd.read_csv('treino.csv')


In [2]:
treino.drop(['id', 'origem','nome'], axis = 1, inplace = True)

In [3]:
treino.head()

,sexo,idade,salario,credito,tempo,regiao,estadocivil,educacao,nprodutos,pagamento,maxatraso,cliente
0,masculino,41.0,44249.0,704.3,54,sul,casado(a),fundamental,1,cartão de débito,10,1
1,masculino,27.0,60347.0,457.1,54,sul,divorciado(a),graduado,0,pix,7,0
2,feminino,40.0,75920.0,552.1,59,sudeste,casado(a),graduado,1,cartão de crédito,0,1
3,masculino,29.0,68066.0,285.5,21,norte,viúvo(a),fundamental,2,cartão de débito,0,0
4,masculino,30.0,76392.0,NaN,20,sudeste,casado(a),médio,3,boleto,6,0


In [4]:
treino.shape

(7391, 12)

In [5]:
treino = treino.dropna()

In [7]:
#padronização das variáveis idade, tempo, maxatraso e crédito
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#Padronização
treino[['idade', 'tempo', 'credito', 'salario']] = sc.fit_transform(treino[['idade', 'tempo', 'credito', 'salario']])

In [8]:
variaveis_cat = ['sexo','regiao','estadocivil','educacao','nprodutos','pagamento']
treino = pd.get_dummies(treino, columns = variaveis_cat, drop_first = True)
treino.head()

,idade,salario,credito,tempo,maxatraso,cliente,sexo_masculino,regiao_nordeste,regiao_norte,regiao_sudeste,...,nprodutos_1,nprodutos_2,nprodutos_3,nprodutos_4,nprodutos_5,nprodutos_6,pagamento_cartão de crédito,pagamento_cartão de débito,pagamento_pix,pagamento_transferência
0,0.527451,-1.033449,1.327074,0.860135,10,1,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1,-0.984919,-0.196131,-0.400999,0.860135,7,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0.419424,0.613880,0.263107,1.102144,0,1,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
3,-0.768867,0.205364,-1.600583,-0.737126,0,0,1,0,1,0,...,0,1,0,0,0,0,0,1,0,0
5,-0.660840,-0.658481,-0.988907,1.828172,5,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [10]:
def MachineLearning(X,y):

    #X = dados.drop('Exited', axis = 1)
    #y = dados['Exited']

    lista_de_medidas = ['accuracy', 'recall', 'precision', 'balanced_accuracy', 'f1']
    nome_das_medidas = ['acurácia', 'sensibilidade', 'precisão', 'eficiência', 'f1-score']

    lista_de_modelos = [LogisticRegression(),
                            DecisionTreeClassifier(max_depth = 3),
                            DecisionTreeClassifier(max_depth = 5),
                            DecisionTreeClassifier(max_depth = 7),
                            KNeighborsClassifier(n_neighbors = 5),
                            KNeighborsClassifier(n_neighbors = 15),
                            KNeighborsClassifier(n_neighbors = 25),
                            BaggingClassifier(),
                            RandomForestClassifier(n_estimators=50, max_depth = 5),
                            RandomForestClassifier(n_estimators=50, max_depth = 7),
                            RandomForestClassifier(n_estimators=100, max_depth = 5),
                            RandomForestClassifier(bootstrap=False, max_depth=10, max_features='sqrt',
                            min_samples_leaf=3, n_estimators=150),
                            XGBClassifier(learning_rate = 0.5, max_depth = 8, n_estimators = 400, eval_metric = 'auc',
                            use_label_encoder=False, objective = 'binary:logistic'),
                            XGBClassifier(learning_rate = 0.5, max_depth = 8, n_estimators = 400, eval_metric = 'auc',
                            use_label_encoder=False, objective = 'binary:logistic'),
                            SVC(),
                            GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,
                                                       max_depth=8,max_features='sqrt',subsample=0.8)
                            
                           ]

    nome_dos_modelos = ['Regressão Logística',
                            'Árvore (prof = 3)',
                            'Árvore (prof = 5)',
                            'Árvore (prof = 7)',
                            '5-NN',
                            '15-NN',
                            '25-NN',
                            'Bagging',
                            'Random Forest (arvs = 50, prof = 5)',
                            'Random Forest (arvs = 50, prof = 7)',
                            'Random Forest (arvs = 100, prof = 5)', 
                            'Random Forest (arvs = 150, prof = 10)',
                            'XGB (0.5, 8, 400)',
                            'XGB (0.5, 6, 200)',
                               'SVC',
                               'Gradiente']

    resultados0 = {}

    validacao = StratifiedKFold(n_splits = 10)


    for i in range(len(lista_de_modelos)):
        print('Rodando modelo: ' + nome_dos_modelos[i])
        accs_vc = cross_validate(lista_de_modelos[i], X, y, cv = validacao, scoring = lista_de_medidas)

        acc = accs_vc['test_accuracy'].mean()
        sen = accs_vc['test_recall'].mean()
        vpp = accs_vc['test_precision'].mean()
        bac = accs_vc['test_balanced_accuracy'].mean()
        f1s = accs_vc['test_f1'].mean()

        resultados0[nome_dos_modelos[i]] = [acc, sen, vpp, f1s, bac]
        
    resultados = pd.DataFrame(resultados0, index = nome_das_medidas).T

    return resultados

In [11]:
X = treino.drop('cliente', axis = 1)
y = treino['cliente']

In [12]:
MachineLearning(X,y)

Rodando modelo: Regressão Logística


c:\python\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\python\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_L

Rodando modelo: Árvore (prof = 3)
Rodando modelo: Árvore (prof = 5)
Rodando modelo: Árvore (prof = 7)
Rodando modelo: 5-NN
Rodando modelo: 15-NN
Rodando modelo: 25-NN
Rodando modelo: Bagging
Rodando modelo: Random Forest (arvs = 50, prof = 5)
Rodando modelo: Random Forest (arvs = 50, prof = 7)
Rodando modelo: Random Forest (arvs = 100, prof = 5)
Rodando modelo: Random Forest (arvs = 150, prof = 10)
Rodando modelo: XGB (0.5, 8, 400)
Rodando modelo: XGB (0.5, 6, 200)
Rodando modelo: SVC
Rodando modelo: Gradiente


,acurácia,sensibilidade,precisão,eficiência,f1-score
Regressão Logística,0.842533,0.759355,0.818314,0.787217,0.826883
Árvore (prof = 3),0.734594,0.724932,0.635363,0.676905,0.732785
Árvore (prof = 5),0.768431,0.589071,0.755008,0.660811,0.734681
Árvore (prof = 7),0.807372,0.675681,0.793180,0.729144,0.782592
5-NN,0.693006,0.523587,0.618564,0.566624,0.661116
15-NN,0.713422,0.471921,0.684062,0.558205,0.667973
25-NN,0.719849,0.462571,0.706649,0.558737,0.671434
Bagging,0.817769,0.678637,0.816827,0.740726,0.791585
"Random Forest (arvs = 50, prof = 5)",0.779206,0.482802,0.893764,0.626065,0.723444
"Random Forest (arvs = 50, prof = 7)",0.818904,0.611200,0.881636,0.721494,0.779813
